# Построение классификатора на базе данных IRIS

In [81]:
from sklearn.datasets import load_iris

import sklearn.model_selection
import tensorflow as tf

## Загрузка данных

In [82]:
iris = load_iris()
X = iris.data
y = iris.target
print(X.shape, y.shape)

(150, 4) (150,)


In [83]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(112, 4) (112,) (38, 4) (38,)


## Параметры обучения

In [84]:
tf.compat.v1.reset_default_graph()

flowers = tf.feature_column.numeric_column('flower', (4,))

batch_size_num = 5
n_classes = 10
feature_column = 'flower'

## Модели

In [85]:
model_1 = tf.estimator.DNNClassifier(feature_columns=[flowers], n_classes=n_classes, hidden_units=[1024, 512, 256])
model_2 = tf.estimator.DNNClassifier(feature_columns=[flowers], n_classes=n_classes, hidden_units=[300, 100])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\imsol\\AppData\\Local\\Temp\\tmp4gxdh7n4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using default config.
INFO:ten

## Вспомогательные функции

In [86]:
def input_evaluation_set():
    dataset = tf.data.Dataset.from_tensor_slices(({feature_column: X_train}, y_train.astype(int)))
    return dataset.shuffle(batch_size_num).repeat().batch(batch_size_num)

In [87]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features = dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

## Обучение моделей

In [88]:
model_1.train(input_fn=input_evaluation_set, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\imsol\AppData\Local\Temp\tmp4gxdh7n4\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 2.0858655, step = 0
INFO:tensorflow:global_step/sec: 682.713
INFO:tensorflow:loss = 0.7967871, step = 100 (0.147 sec)
INFO:tensorflow:global_step/sec: 806.352
INFO:tensorflow:loss = 0.6464727, step = 200 (0.124 sec)
INFO:tensorflow:global_step/sec: 777.476
INFO:tensorflow:loss = 0.38555253, step = 300 (0.129 sec)
INFO:tensorflow:global_step/sec: 726.283
INFO:tensorflow:loss = 0.36815518, step = 400 (0.138 sec)
INFO:tensorflow:global_step/sec: 994.229
INFO:tensorflow:loss = 0.5574428

In [89]:
model_2.train(input_fn=input_evaluation_set, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\imsol\AppData\Local\Temp\tmpkp5sxua8\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 2.7899423, step = 0
INFO:tensorflow:global_step/sec: 1442.39
INFO:tensorflow:loss = 0.9621269, step = 100 (0.070 sec)
INFO:tensorflow:global_step/sec: 2497.77
INFO:tensorflow:loss = 1.0354253, step = 200 (0.040 sec)
INFO:tensorflow:global_step/sec: 2497.64
INFO:tensorflow:loss = 0.8655852, step = 300 (0.040 sec)
INFO:tensorflow:global_step/sec: 2560.22
INFO:tensorflow:loss = 0.7277392, step = 400 (0.039 sec)
INFO:tensorflow:global_step/sec: 2559.95
INFO:tensorflow:loss = 0.61465156,

## Получение результатов оценки

In [90]:
print(model_1.evaluate(input_fn=lambda: eval_input_fn({'flower': X_test}, y_test.astype(int), batch_size_num)))
print(model_2.evaluate(input_fn=lambda: eval_input_fn({'flower': X_test}, y_test.astype(int), batch_size_num)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-05-13T19:31:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\imsol\AppData\Local\Temp\tmp4gxdh7n4\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.13478s
INFO:tensorflow:Finished evaluation at 2024-05-13-19:31:15
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.9736842, average_loss = 0.26503313, global_step = 1000, loss = 0.25730395
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\imsol\AppData\Local\Temp\tmp4gxdh7n4\model.ckpt-1000
{'accuracy': 0.9736842, 'average_loss': 0.26503313, 'loss': 0.25730395, 'global_step': 1000}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-05-13T19:31:15
INFO:tensorflow:Graph was finalized.
INFO:tensor